# Random Forest from the scratch (using dataset Adult from UCI)

A modifield version of a modifield version of:
Decision Tree from the Scratch, Rakend Dubba (Computational Engineer | Data Scientist).
*Source:* https://medium.com/@rakendd/decision-tree-from-scratch-9e23bcfb4928.

Implemented random split (for feature, not the criteria split, based in entropy) and compare with not random (the same in *exemplo2*, basic RF)

Done:

1) Using Bagging, get a group of $N_B$ random samples ($x_i, i = 1,... ,N_B$) with replacement for each three, for all $M$ trees.

2) Each tree with a maximum limit of $s_{MAX}$ splitlevels. Why there categorical features with more then two values, each level may have more then two nodes. *There is implemented the limit of splits trough each way, i.e., fallowing the same sequence till limit.

5) The ensembling model is based in voting, may possible to use both majority or soft. The schoice is made when using the predict funcion. *Rather that, it is use only soft voting.

The final version, we will have:

3) Each tree receive $K = s_{MAX}$ random features from all $p$ features.

4) There are two alternatives for splitting with numeric features: using entropy criteria and random splitting between max/min values. For categorical features, all values receive a node.

In [1]:
import re
import numpy as np
import pandas as pd
eps = np.finfo(float).eps
from numpy import log2 as log
from tabulate import tabulate as tb
from anytree import Node, RenderTree
from anytree import search as anys
from anytree.exporter import DotExporter
from IPython.display import Image

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


##### Load dataset:

In [2]:
features = ["Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Marital Status",
            "Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss",
            "Hours per week", "Country", "Target"]
train_data = pd.read_csv(
    #"https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    "adult.data",
    names=features,
    sep=r'\s*,\s*',
    engine='python',
    na_values="?").dropna()

Target = 'Target'
Labels = train_data.Target.unique()
counts = train_data.Target.value_counts()
print(counts)

test_data = pd.read_csv(
    #"https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test",
    "adult.test_fix",
    names=features,
    sep=r'\s*,\s*',
    skiprows=[0],
    engine='python',
    na_values="?").dropna()

Labels = test_data.Target.unique()
counts = test_data.Target.value_counts()
print(counts)


<=50K    22654
>50K      7508
Name: Target, dtype: int64
<=50K    11360
>50K      3700
Name: Target, dtype: int64


In [43]:
def find_entropy(df):
    entropy = 0
    values = df[Target].unique()
    for value in values:
        temp = df[Target].value_counts()[value]/len(df[Target])
        entropy += -temp*np.log2(temp)
    return entropy

def find_entropy_attribute(df,attribute):
    
    if not np.issubdtype(df[attribute].dtype, np.number):   
        return find_entropy_attribute_not_number(df,attribute), None
    else:
        return find_entropy_attribute_number(df,attribute)
          
        
def find_entropy_attribute_not_number(df,attribute):
    target_variables = df[Target].unique()  #This gives all 'Yes' and 'No'
    variables = df[attribute].unique()    #This gives different features in that attribute (like 'Hot','Cold' in Temperature)
    entropy2 = 0
    for variable in variables:
        entropy = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute]==variable][df[Target] ==target_variable])
            den = len(df[attribute][df[attribute]==variable])
            fraction = num/(den+eps)
            entropy += -fraction*log(fraction+eps)
        entropy2 += -(den/len(df))*entropy
    return abs(entropy2)


def find_entropy_attribute_number(df,attribute):
    target_variables = df[Target].unique()  #This gives all 'Yes' and 'No'
    variables = df[attribute].unique()    #This gives different features in that attribute (like 'Hot','Cold' in Temperature)
    variables.sort()
    if len(variables)>2:
        variables = variables[1:-1]
        vk3 = variables[0]
        entropy3 = 0
    else:
        vk3 = variables[0]
        entropy3 = np.Inf
    
    for vk in variables:
        entropy = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute]<=vk][df[Target] ==target_variable])
            den = len(df[attribute][df[attribute]<=vk])
            fraction = num/(den+eps)
            entropy += -fraction*log(fraction+eps)
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute]>vk][df[Target] ==target_variable])
            den = len(df[attribute][df[attribute]>vk])
            fraction = num/(den+eps)
            entropy += -fraction*log(fraction+eps)
        entropy2 = (den/len(df))*abs(entropy)
        #print(str(entropy2)+"|"+str(vk))
        if entropy2>entropy3:
            entropy3 = entropy2
            vk3 = vk
    return abs(entropy3),vk3

def find_winner(df,rfs_par=False):
    if not rfs_par:
        IG = []
        vk = list()
        for key in df.columns.difference([Target]):
            temp,temp2 = find_entropy_attribute(df,key)
            vk.append(temp2)
            IG.append(find_entropy(df)-temp)
        return df.columns.difference([Target])[np.argmax(IG)], vk[np.argmax(IG)]
    else:
        key = df.columns.difference([Target])[np.random.randint(len(df.columns.difference([Target]))-1)]
        temp,vk = find_entropy_attribute(df,key)
        return key, vk
        

def print_result_node(node,value,classe,prob):
    print(node +' : '+value+' : '+classe+' ('+str(prob)+')')

def buildtree(df,tree=None, mytree=None, T_pro=0.9, T_pro_num=0.6,total_splits=10,splits=1,rfs=False):
    
    def ramificatree(Thd,ss):
        if (len(clValue)==1):
            tree[node][value] = {}
            tree[node][value]['Class'] = clValue[0]
            tree[node][value]['Prob'] = 1.0
            #print_result_node(node,value,clValue[0],1)
        else:
            prob = counts.max() / counts.sum()
            if (prob>=Thd)or(splits>=total_splits):
                tree[node][value] = {}
                tree[node][value]['Class'] = clValue[counts.argmax()]
                tree[node][value]['Prob'] = prob
                #print_result_node(node,value,clValue[counts.argmax()],prob)
            else:
                ss += 1
                tree[node][value] = buildtree(subtable,splits=ss)
                #print(node +' : '+value+' : *')

    #print(find_winner(df))
    #formata_dados(dados)
    node,vk = find_winner(df,rfs_par=rfs)

    if tree is None:
        tree={}
        tree[node] = {}

    if vk is None:
        attValue = np.unique(df[node])
        for value in attValue:

            subtable = df[df[node] == value].reset_index(drop=True)
            clValue,counts = np.unique(subtable[Target],return_counts=True)

            splits += 1
            ramificatree(T_pro,ss=splits)
    else:
        
        if (len(df[node][df[node] <= vk].unique())>0) and (len(df[node][df[node] > vk].unique())>0):
           
            # >vk
            value = node+' >'+str(vk)
            subtable = df[df[node] > vk].rename(columns = {node:value}).reset_index(drop=True)
            clValue,counts = np.unique(subtable[Target],return_counts=True)            
            if (len(subtable[value].unique())==1) and (len(clValue)>1):
                tree[node][value] = {}
                tree[node][value]['Class'] = clValue[counts.argmax()]
                prob = counts.max() / counts.sum()
                tree[node][value]['Prob'] = prob
                #print_result_node(node,value,clValue[counts.argmax()],prob)
            else:
                splits += 1
                ramificatree(T_pro_num,ss=splits)
            clValue_antes = clValue[0]
            value_antes = value
            # <=vk
            value = node+' <='+str(vk)
            subtable = df[df[node] <= vk].rename(columns = {node:value}).reset_index(drop=True)
            clValue,counts = np.unique(subtable[Target],return_counts=True)
            if ((len(subtable[value].unique())==1) and (len(clValue)>1)):
                tree[node][value] = {}
                tree[node][value]['Class'] = clValue[counts.argmax()]
                prob = counts.max() / counts.sum()
                tree[node][value]['Prob'] = prob
                #print_result_node(node,value,clValue[counts.argmax()],prob)
            else:
                splits += 1
                ramificatree(T_pro_num,ss=splits)

        else:
            df[node] = df[node].astype(str)
            buildtree(df)
        
    return tree


# Only to see

def print_tree(arg):
    for pre, fill, node in RenderTree(arg):
        print("%s%s" % (pre, node.name))
        
def converte_para_anytree(tree,node=None,mytree=None):
    
    if node is None:
        temp = list(tree.keys())
        node = temp[0]
        mytree = {}
        mytree[node] = Node(node)
        converte_para_anytree(tree,node,mytree)
    else:
        tree = tree[node]
        if not isinstance(tree, str):
            childs = list(tree.keys())
            for child in childs:
                if (list(tree[child])[0] == 'Class'):
                    temp = mytree[node]
                    mytree[child] = Node(child, parent=temp, target=tree[child]['Class'], prob=tree[child]['Prob'])
                else:
                    temp = mytree[node]
                    mytree[child] = Node(child, parent=temp)
                    converte_para_anytree(tree,child,mytree)
        else:
            mytree[node] = 'Fim'
                
    return mytree

#anys.findall_by_attr(mytree['Taste'], name="target", value='Yes')

def mostra_tree(tree):
    mytree = converte_para_anytree(tree)

    temp = list(tree.keys())
    root = temp[0]
    mytree[root]

    for pre, fill, node in RenderTree(mytree[root]):
        txt_node = str(node)
        m = re.search('prob\=\d+\.\d+', txt_node)
        if  Labels[0] in txt_node:
            if not m is None:
                print("%s%s" % (pre, node.name+': '+Labels[0]+' ('+m.group()[5:]+')'))
            else:
                print("%s%s" % (pre, node.name+': '+Labels[0]+' (?)'))
        elif Labels[1] in txt_node:
            if not m is None:
                print("%s%s" % (pre, node.name+': '+Labels[1]+' ('+m.group()[5:]+')'))
            else:
                print("%s%s" % (pre, node.name+': '+Labels[1]+' (?)'))
        else:
            print("%s%s" % (pre, node.name))
    
    
def mostra_tree_graph(tree, largura=None, altura=None):
    mytree = converte_para_anytree(tree)

    temp = list(tree.keys())
    root = temp[0]
    mytree[root]
    DotExporter(mytree[root]).to_picture("tree.png")
    return Image(filename='tree.png', width=largura, height=altura) 


def predict(inst,tree):

    for node in tree.keys():     

        if ('<=' in str(tree[node].keys())):

            childs = list(tree[node].keys())

            if ('<=' in childs[1]):
                temp = childs[1]
                childs[1] = childs[0]
                childs[0] = temp

            vk = float(childs[1].split('>')[1])
            
            if ('>' in node):
                valor = float(str(inst[node.split('>')[0][:-1]]))
            elif ('<=' in node):
                valor = float(str(inst[node.split('<')[0][:-1]]))
            else:
                valor = float(str(inst[node]))
            if (valor > vk):
                tree = tree[node][childs[1]]
                prediction = None
                prob = None
                if (list(tree)[0] != 'Class'):
                    prediction,prob = predict(inst, tree)
                else:
                    prediction = tree['Class']
                    prob = tree['Prob']
                    break;
            else:
                tree = tree[node][childs[0]]
                prediction = None
                prob = None
                if (list(tree)[0] != 'Class'):
                    prediction,prob = predict(inst, tree)
                else:
                    prediction = tree['Class']
                    prob = tree['Prob']
                    break;
            
        else:
            value = str(inst[node])
            if value in tree[node].keys():
                tree = tree[node][value]
                prediction = None
                prob = None
                if (list(tree)[0] != 'Class'):
                    prediction,prob = predict(inst, tree)
                else:
                    prediction = tree['Class']
                    prob = tree['Prob']
                    break;
            else:
                prediction = 'Not exists node: '+value
                prob = 0

    
    return prediction, prob

def predict_forest(arg,forest):
    prob_yes = 0
    prob_no = 0
    for tree in forest:
        result = predict(arg,tree)
        if (result[0] == arg.Target):
            prob_yes += result[1]
        else:
            prob_no += 1-result[1]
    return prob_yes, prob_no

def test_step_prob(arg,tree):
    P = 0;
    S = 0
    for i in range(0,len(arg)):
        S += (predict(arg.iloc[i],tree)[0] == arg.iloc[i].Target)*1
        P += predict(arg.iloc[i],tree)[1]

    S = S / len(arg)
    P = P / len(arg)
    print(str(S)+' ('+str(P)+')')

def test_step(arg,tree):
    NO = 0;
    YES = 0
    for i in range(0,len(arg)):
        if (predict(arg.iloc[i],tree)[0] == arg.iloc[i].Target):
            YES += 1
        else:
            NO += 1
    YES = YES / len(arg)
    NO = NO / len(arg)
    #print("YES: "+str(YES)+'. NO: '+str(NO)+'.')
    return YES,NO

def test_step_forest(arg,forest):
    YES = 0
    for i in range(0,len(arg)):
        result = predict_forest(arg.loc[i],forest)
        if result[0]>result[1]:
            YES += 1
    YES = YES / len(arg)
    return YES

# Bagging functions:

def formata_dados(dados):
    for chave in dados.keys():
        if not np.issubdtype(dados[chave].dtype, np.number):
            dados[chave] = dados[chave].astype(str)
        elif (len(dados[chave].unique())<5):
            dados[chave] = dados[chave].astype(str)
    return dados

def amostra_dados(dados,n_samples):
    dados2 = dados.loc[dados[Target]==Labels[0]].sample(int(n_samples/2))
    dados2 = dados2.append(dados.loc[dados[Target]==Labels[1]].sample(int(n_samples/2)), ignore_index=True).reset_index(drop=True)
    return formata_dados(dados2)

In [62]:
n_samples=100
forest = list()
forest_rfs = list()
M = 400
for m in range(0,M):
    print(str(m+1)+'/'+str(M), end='\r')
    train_bag = amostra_dados(train_data,n_samples)    
    forest.append(buildtree(train_bag,T_pro=0.8, T_pro_num=0.8))
    forest_rfs.append(buildtree(train_bag,T_pro=0.8, T_pro_num=0.8, rfs=True))

In [ ]:

T = 100
values_forest = np.empty((T,1))
values_forest_rfs = np.empty((T,1))
for r in range(0,T):
    print(str(r+1)+'/'+str(T), end='\r')
    n_samples_test = 1000
    test_bag = amostra_dados(test_data,n_samples_test)
    values_forest[r] = test_step_forest(test_bag,forest)
    values_forest_rfs[r] = test_step_forest(test_bag,forest_rfs)
    
mean_values_forest = values_forest.mean()
mean_values_forest_rfs = values_forest_rfs.mean()
std_values_forest = values_forest.std()
std_values_forest_rfs = values_forest_rfs.std()



In [ ]:
print(tb([['Forest ', "{:.2f}".format(mean_values_forest)+" +-{:.2f}".format(std_values_forest)], \
          ['Forest (RFS) ', "{:.2f}".format(mean_values_forest_rfs)+" +-{:.2f}".format(std_values_forest_rfs)]], \
          headers=["Method", "Precision (%)"], tablefmt='orgtbl'))

In [ ]:
size_tree = np.empty((M,1))
m=0
for tree in values_forest_rfs:
    size_tree[m] = len(str(tree))
    m+=1

In [ ]:
test_step(test_bag,forest[size_tree.argmin()])

In [ ]:
mostra_tree_graph(forest[size_tree.argmin()])

In [ ]:
mostra_tree(forest[size_tree.argmin()])

In [ ]:
test_step(test_bag,forest[size_tree.argmax()])

In [ ]:
mostra_tree_graph(forest[size_tree.argmax()])

In [ ]:
mostra_tree(forest[size_tree.argmax()])